# Data Prep for Neo4j Import

## 1. Load Data, Clean NaN Values, and Remove Unnecessary Columns

In [1]:
import neo4j
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("2015-2021 CFB Recruiting Players data.csv", engine='python')
print(df.shape)
df.head()

(27179, 19)


,id,athleteId,recruitType,year,ranking,name,school,committedTo,position,height,weight,stars,rating,city,stateProvince,country,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode
0,66928,3915192.0,HighSchool,2015,1.0,Trenton Thompson,Westover,Georgia,DT,74.0,313.0,5,0.9992,Albany,GA,USA,31.578206,-84.155681,13095.0
1,31860,NaN,HighSchool,2015,1.0,Trent Thompson,Westover,Georgia,DT,74.5,313.0,5,0.9991,Albany,GA,USA,31.578206,-84.155681,13095.0
2,31861,-1009710.0,HighSchool,2015,2.0,Martez Ivey,Apopka,Florida,OT,77.5,275.0,5,0.9990,Apopka,FL,USA,28.677968,-81.511521,12095.0
3,31862,3916922.0,HighSchool,2015,3.0,Byron Cowart,Armwood,Auburn,SDE,76.0,250.0,5,0.9987,Seffner,FL,USA,27.998541,-82.274884,12057.0
4,31863,3912545.0,HighSchool,2015,4.0,Iman Marshall,Long Beach Poly,USC,CB,73.0,190.0,5,0.9985,Long Beach,CA,USA,33.769016,-118.191605,6037.0


In [3]:
df.dtypes

id                          int64
athleteId                 float64
recruitType                object
year                        int64
ranking                   float64
name                       object
school                     object
committedTo                object
position                   object
height                    float64
weight                    float64
stars                       int64
rating                    float64
city                       object
stateProvince              object
country                    object
hometownInfo.latitude     float64
hometownInfo.longitude    float64
hometownInfo.fipsCode     float64
dtype: object

In [4]:
df.isnull().sum(axis = 0)

id                            0
athleteId                 11704
recruitType                   0
year                          0
ranking                     360
name                          0
school                      275
committedTo                5524
position                     18
height                       51
weight                       55
stars                         0
rating                        0
city                        298
stateProvince               300
country                     196
hometownInfo.latitude       380
hometownInfo.longitude      380
hometownInfo.fipsCode       391
dtype: int64

In [5]:
df.nunique(axis=0)

id                        27179
athleteId                 15182
recruitType                   1
year                          7
ranking                    4257
name                      25912
school                     5291
committedTo                 271
position                     22
height                       57
weight                      244
stars                         5
rating                     2186
city                       3034
stateProvince                60
country                      17
hometownInfo.latitude      3611
hometownInfo.longitude     3611
hometownInfo.fipsCode      1571
dtype: int64

In [6]:
df = df.drop(['athleteId', 'recruitType','country'], axis=1)

In [7]:
df['year'].value_counts()

2017    4359
2020    4307
2019    4165
2016    4053
2018    3950
2015    3608
2021    2737
Name: year, dtype: int64

In [8]:
df1 = df.drop_duplicates(
    subset=['year','ranking'], 
    keep="first").reset_index(drop=True)
print(df1.shape)
df1.head()

(26403, 16)


,id,year,ranking,name,school,committedTo,position,height,weight,stars,rating,city,stateProvince,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode
0,66928,2015,1.0,Trenton Thompson,Westover,Georgia,DT,74.0,313.0,5,0.9992,Albany,GA,31.578206,-84.155681,13095.0
1,31861,2015,2.0,Martez Ivey,Apopka,Florida,OT,77.5,275.0,5,0.9990,Apopka,FL,28.677968,-81.511521,12095.0
2,31862,2015,3.0,Byron Cowart,Armwood,Auburn,SDE,76.0,250.0,5,0.9987,Seffner,FL,27.998541,-82.274884,12057.0
3,31863,2015,4.0,Iman Marshall,Long Beach Poly,USC,CB,73.0,190.0,5,0.9985,Long Beach,CA,33.769016,-118.191605,6037.0
4,31864,2015,5.0,Derwin James,Haines City Senior,Florida State,S,74.0,201.0,5,0.9981,Auburndale,FL,28.107088,-81.803580,12105.0


In [9]:
df1.isnull().sum(axis = 0)

id                           0
year                         0
ranking                      3
name                         0
school                      30
committedTo               5361
position                    10
height                      46
weight                      50
stars                        0
rating                       0
city                       125
stateProvince              127
hometownInfo.latitude      204
hometownInfo.longitude     204
hometownInfo.fipsCode      213
dtype: int64

In [10]:
df1['year'].value_counts()

2017    4252
2019    4090
2016    3978
2020    3934
2018    3887
2015    3547
2021    2715
Name: year, dtype: int64

In [11]:
df1[['committedTo']] = df1[['committedTo']].fillna("Uncommitted")

In [12]:
# pd.set_option('display.max_rows',500)

df1_group = df1.groupby(['committedTo','year']).size().unstack(fill_value=0)
df1_group.head()

year,2015,2016,2017,2018,2019,2020,2021
committedTo,,,,,,,
Abilene Christian,2,8,3,0,0,0,0
Air Force,22,40,59,26,51,33,47
Akron,14,7,20,14,22,17,16
Alabama,24,22,27,21,27,24,26
Alabama A&M,0,1,1,1,1,0,1


In [13]:
# check uncommitted recruit totals
df1_group.query('committedTo== "Uncommitted"')

year,2015,2016,2017,2018,2019,2020,2021
committedTo,,,,,,,
Uncommitted,549,763,958,829,1042,996,224


In [14]:
df1.isnull().sum(axis = 0)

id                          0
year                        0
ranking                     3
name                        0
school                     30
committedTo                 0
position                   10
height                     46
weight                     50
stars                       0
rating                      0
city                      125
stateProvince             127
hometownInfo.latitude     204
hometownInfo.longitude    204
hometownInfo.fipsCode     213
dtype: int64

In [15]:
# drop rows containing NaN values for selected columns

df1.dropna(subset=['school','ranking', 'position','city', 'stateProvince',
                   'hometownInfo.latitude','hometownInfo.longitude','hometownInfo.fipsCode'], inplace=True)

print(df1.shape)
df1.head()

(26159, 16)


,id,year,ranking,name,school,committedTo,position,height,weight,stars,rating,city,stateProvince,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode
0,66928,2015,1.0,Trenton Thompson,Westover,Georgia,DT,74.0,313.0,5,0.9992,Albany,GA,31.578206,-84.155681,13095.0
1,31861,2015,2.0,Martez Ivey,Apopka,Florida,OT,77.5,275.0,5,0.9990,Apopka,FL,28.677968,-81.511521,12095.0
2,31862,2015,3.0,Byron Cowart,Armwood,Auburn,SDE,76.0,250.0,5,0.9987,Seffner,FL,27.998541,-82.274884,12057.0
3,31863,2015,4.0,Iman Marshall,Long Beach Poly,USC,CB,73.0,190.0,5,0.9985,Long Beach,CA,33.769016,-118.191605,6037.0
4,31864,2015,5.0,Derwin James,Haines City Senior,Florida State,S,74.0,201.0,5,0.9981,Auburndale,FL,28.107088,-81.803580,12105.0


In [16]:
df1.isnull().sum(axis = 0)

id                         0
year                       0
ranking                    0
name                       0
school                     0
committedTo                0
position                   0
height                    43
weight                    46
stars                      0
rating                     0
city                       0
stateProvince              0
hometownInfo.latitude      0
hometownInfo.longitude     0
hometownInfo.fipsCode      0
dtype: int64

In [17]:
# check recruits who do not have height listed
df1[df1['height'].isna()].head()

,id,year,ranking,name,school,committedTo,position,height,weight,stars,rating,city,stateProvince,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode
1328,66964,2015,1331.0,Ronique Robinson,Booker T. Washington,Uncommitted,WDE,NaN,NaN,3,0.8333,Miami,FL,25.774266,-80.193659,12086.0
2962,67114,2015,2969.0,Kiki Hill,Goliad,Uncommitted,WR,NaN,NaN,2,0.7667,Goliad,TX,28.668656,-97.390912,48175.0
5059,36638,2016,1513.0,Kailen Guillory,Ben Davis,Western Michigan,WDE,NaN,NaN,3,0.8240,Indianapolis,IN,39.768333,-86.158350,18097.0
5535,37099,2016,1989.0,Taeshon Trotter,Cass Technical,Ohio,OT,NaN,NaN,3,0.8039,Detroit,MI,42.331551,-83.046640,26163.0
5540,37104,2016,1994.0,Kee Whetzel,Countryside,Oregon State,WR,NaN,186.0,3,0.8036,Clearwater,FL,27.965853,-82.800103,12103.0


In [18]:
# impute mean height by position to missing height values
df1.height = df1.groupby('position')['height'].apply(lambda x: x.fillna(x.mean()))
df1.height = df1.height.fillna(df1.height.mean())

In [19]:
# impute mean weight by position to missing weight values
df1.weight = df1.groupby('position')['weight'].apply(lambda x: x.fillna(x.mean()))
df1.weight = df1.weight.fillna(df1.weight.mean())

In [20]:
# check for one of the players without height/weight listed 

df1[df1['name'] == "Kiki Hill"]

,id,year,ranking,name,school,committedTo,position,height,weight,stars,rating,city,stateProvince,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode
2962,67114,2015,2969.0,Kiki Hill,Goliad,Uncommitted,WR,72.887123,182.179795,2,0.7667,Goliad,TX,28.668656,-97.390912,48175.0


In [21]:
# ensure all rows have values

df1.isnull().sum(axis = 0)

id                        0
year                      0
ranking                   0
name                      0
school                    0
committedTo               0
position                  0
height                    0
weight                    0
stars                     0
rating                    0
city                      0
stateProvince             0
hometownInfo.latitude     0
hometownInfo.longitude    0
hometownInfo.fipsCode     0
dtype: int64

## 2. Add County names based on FIPS code

In [22]:
# load new dataframe with fips codes and county names

url ='https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt'
df_5 = pd.read_csv(url, header=None, sep=",",names=['state','fips1','fips2','county','E'],dtype=str)
df_5.head()

,state,fips1,fips2,county,E
0,AL,01,001,Autauga County,H1
1,AL,01,003,Baldwin County,H1
2,AL,01,005,Barbour County,H1
3,AL,01,007,Bibb County,H1
4,AL,01,009,Blount County,H1


In [23]:
df_5['hometownInfo.fipsCode'] = df_5['fips1'] + df_5['fips2']
df_5.head()

,state,fips1,fips2,county,E,hometownInfo.fipsCode
0,AL,01,001,Autauga County,H1,01001
1,AL,01,003,Baldwin County,H1,01003
2,AL,01,005,Barbour County,H1,01005
3,AL,01,007,Bibb County,H1,01007
4,AL,01,009,Blount County,H1,01009


In [24]:
df6 = df_5[['hometownInfo.fipsCode','county']].copy()
df6['hometownInfo.fipsCode'] = pd.to_numeric(df6['hometownInfo.fipsCode'])
df6.head()

,hometownInfo.fipsCode,county
0,1001,Autauga County
1,1003,Baldwin County
2,1005,Barbour County
3,1007,Bibb County
4,1009,Blount County


In [25]:
# merge county names onto existing dataframe

df2 = df1.merge(df6, on='hometownInfo.fipsCode', how='left')
df2.head(10)

,id,year,ranking,name,school,committedTo,position,height,weight,stars,rating,city,stateProvince,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode,county
0,66928,2015,1.0,Trenton Thompson,Westover,Georgia,DT,74.0,313.0,5,0.9992,Albany,GA,31.578206,-84.155681,13095.0,Dougherty County
1,31861,2015,2.0,Martez Ivey,Apopka,Florida,OT,77.5,275.0,5,0.9990,Apopka,FL,28.677968,-81.511521,12095.0,Orange County
2,31862,2015,3.0,Byron Cowart,Armwood,Auburn,SDE,76.0,250.0,5,0.9987,Seffner,FL,27.998541,-82.274884,12057.0,Hillsborough County
3,31863,2015,4.0,Iman Marshall,Long Beach Poly,USC,CB,73.0,190.0,5,0.9985,Long Beach,CA,33.769016,-118.191605,6037.0,Los Angeles County
4,31864,2015,5.0,Derwin James,Haines City Senior,Florida State,S,74.0,201.0,5,0.9981,Auburndale,FL,28.107088,-81.803580,12105.0,Polk County
5,31865,2015,6.0,Kahlil McKenzie,Clayton Valley,Tennessee,DT,75.0,339.0,5,0.9968,Concord,CA,37.976852,-122.033562,6013.0,Contra Costa County
6,31866,2015,7.0,CeCe Jefferson,Baker County Senior,Florida,SDE,74.0,275.0,5,0.9951,Glen Saint Mary,FL,30.275791,-82.160669,12003.0,Baker County
7,31867,2015,8.0,Josh Sweat,Oscar Smith,Florida State,WDE,77.0,240.0,5,0.9948,Chesapeake,VA,36.718371,-76.246680,51550.0,Chesapeake city
8,31868,2015,9.0,Kevin Toliver II,Trinity Christian Academy,LSU,CB,74.0,185.0,5,0.9948,Jacksonville,FL,30.332184,-81.655651,12031.0,Duval County
9,31869,2015,10.0,Malik Jefferson,Poteet,Texas,OLB,74.5,215.0,5,0.9929,Mesquite,TX,32.766610,-96.599472,48113.0,Dallas County


In [26]:
df2.isnull().sum(axis = 0)

id                        0
year                      0
ranking                   0
name                      0
school                    0
committedTo               0
position                  0
height                    0
weight                    0
stars                     0
rating                    0
city                      0
stateProvince             0
hometownInfo.latitude     0
hometownInfo.longitude    0
hometownInfo.fipsCode     0
county                    0
dtype: int64

In [27]:
# rename columns for more accurate/clean headers

df2.rename({'school': 'high_school', 'committedTo': 'college_commit', 'stateProvince': 'state',
           'hometownInfo.latitude': 'hometown_lat', 'hometownInfo.longitude': 'hometown_long', 
            'hometownInfo.fipsCode': 'hometown_fips'}, axis=1, inplace=True)

print(df2.shape)
df2.head()

(26159, 17)


,id,year,ranking,name,high_school,college_commit,position,height,weight,stars,rating,city,state,hometown_lat,hometown_long,hometown_fips,county
0,66928,2015,1.0,Trenton Thompson,Westover,Georgia,DT,74.0,313.0,5,0.9992,Albany,GA,31.578206,-84.155681,13095.0,Dougherty County
1,31861,2015,2.0,Martez Ivey,Apopka,Florida,OT,77.5,275.0,5,0.9990,Apopka,FL,28.677968,-81.511521,12095.0,Orange County
2,31862,2015,3.0,Byron Cowart,Armwood,Auburn,SDE,76.0,250.0,5,0.9987,Seffner,FL,27.998541,-82.274884,12057.0,Hillsborough County
3,31863,2015,4.0,Iman Marshall,Long Beach Poly,USC,CB,73.0,190.0,5,0.9985,Long Beach,CA,33.769016,-118.191605,6037.0,Los Angeles County
4,31864,2015,5.0,Derwin James,Haines City Senior,Florida State,S,74.0,201.0,5,0.9981,Auburndale,FL,28.107088,-81.803580,12105.0,Polk County


In [28]:
# check data types 

df2.dtypes

id                  int64
year                int64
ranking           float64
name               object
high_school        object
college_commit     object
position           object
height            float64
weight            float64
stars               int64
rating            float64
city               object
state              object
hometown_lat      float64
hometown_long     float64
hometown_fips     float64
county             object
dtype: object

## 3. Export to CSV to check

In [29]:
# check csv file for errors

df2.to_csv('2015_2021_Recruits2.csv',encoding='utf-8-sig')

## 4. Node Prep: Final Dataframes after Cleaning

In [30]:
# Players Node

player_df = df2[['id','year','name','height','weight']]
print(player_df.shape)
player_df.head()

(26159, 5)


,id,year,name,height,weight
0,66928,2015,Trenton Thompson,74.0,313.0
1,31861,2015,Martez Ivey,77.5,275.0
2,31862,2015,Byron Cowart,76.0,250.0
3,31863,2015,Iman Marshall,73.0,190.0
4,31864,2015,Derwin James,74.0,201.0


In [31]:
# Ratings Node

rating_df = df2[['id','year','ranking','stars','rating']]
print(rating_df.shape)
rating_df.head()

(26159, 5)


,id,year,ranking,stars,rating
0,66928,2015,1.0,5,0.9992
1,31861,2015,2.0,5,0.9990
2,31862,2015,3.0,5,0.9987
3,31863,2015,4.0,5,0.9985
4,31864,2015,5.0,5,0.9981


In [32]:
# Hometown Node

hometown_df = df2[['id','city','hometown_fips','county','state','hometown_lat','hometown_long']]
print(hometown_df.shape)
hometown_df.head()

(26159, 7)


,id,city,hometown_fips,county,state,hometown_lat,hometown_long
0,66928,Albany,13095.0,Dougherty County,GA,31.578206,-84.155681
1,31861,Apopka,12095.0,Orange County,FL,28.677968,-81.511521
2,31862,Seffner,12057.0,Hillsborough County,FL,27.998541,-82.274884
3,31863,Long Beach,6037.0,Los Angeles County,CA,33.769016,-118.191605
4,31864,Auburndale,12105.0,Polk County,FL,28.107088,-81.803580


In [33]:
# Position Node

position_df = df2[['id','position']]
print(position_df.shape)
position_df.head()

(26159, 2)


,id,position
0,66928,DT
1,31861,OT
2,31862,SDE
3,31863,CB
4,31864,S


In [34]:
# School Node

school_df = df2[['id','high_school']]
print(school_df.shape)
school_df.head()

(26159, 2)


,id,high_school
0,66928,Westover
1,31861,Apopka
2,31862,Armwood
3,31863,Long Beach Poly
4,31864,Haines City Senior


In [35]:
# College Node

college_df = df2[['id','college_commit']]
print(college_df.shape)
college_df.head()

(26159, 2)


,id,college_commit
0,66928,Georgia
1,31861,Florida
2,31862,Auburn
3,31863,USC
4,31864,Florida State


In [36]:
# export dataframes 

player_df.to_csv('data/player.csv', index=False)
rating_df.to_csv('data/rating.csv', index=False)
hometown_df.to_csv('data/hometown.csv', index=False)
position_df.to_csv('data/position.csv', index=False)
school_df.to_csv('data/school.csv', index=False)
college_df.to_csv('data/college.csv', index=False)

## 5. Cypher Import Script

In [37]:
CREATE CONSTRAINT ON (n:Player) ASSERT n.id is UNIQUE;
CREATE CONSTRAINT ON (n:Rating) ASSERT n.id is UNIQUE; 
CREATE CONSTRAINT ON (n:Hometown) ASSERT n.id is UNIQUE; 
CREATE CONSTRAINT ON (n:Position) ASSERT n.position is UNIQUE; 
CREATE CONSTRAINT ON (n:School) ASSERT n.high_school is UNIQUE; 
CREATE CONSTRAINT ON (n:College) ASSERT n.college_commit is UNIQUE;

LOAD CSV WITH HEADERS FROM 'file:///player.csv' AS row
CREATE(:Player
{
    `id`: row.`id`,
    `year`: row.`year`,
    `name`: row.`name`,
    `height`: row.`height`,
    `weight`: row.`weight` 
});

LOAD CSV WITH HEADERS FROM 'file:///rating.csv' AS row
MERGE (n:Rating {year: row.year,
    ranking: row.ranking,
    stars: row.stars,
    rating: row.rating}) 
WITH row, n 
MATCH(p:Player {id: row.id}) 
MERGE (p)-[:HAS_RATING]->(n)

LOAD CSV WITH HEADERS FROM 'file:///hometown.csv' AS row
MERGE (n:Hometown {city: row.city,
    hometown_fips: row.hometown_fips,
    county: row.county,
    state: row.state,
    hometown_lat: row.hometown_lat,
    hometown_long: row.hometown_long}) 
WITH row, n 
MATCH(p:Player {id: row.id}) 
MERGE (p)<-[:IS_FROM]-(n)

LOAD CSV WITH HEADERS FROM 'file:///college.csv' AS row
MERGE (n:College {name: row.college_commit}) 
WITH row, n 
MATCH(p:Player {id: row.id}) 
MERGE (p)<-[:COMMITTED_TO]-(n)

LOAD CSV WITH HEADERS FROM 'file:///school.csv' AS row
MERGE (n:School {name: row.school}) 
WITH row, n 
MATCH(p:Player {id: row.id}) 
MERGE (p)<-[:PLAYED_AT]-(n)

LOAD CSV WITH HEADERS FROM 'file:///position.csv' AS row 
MERGE (n:Position {name: row.position}) 
WITH row, n 
MATCH(p:Player {id: row.id}) 
MERGE (p)-[:HAS_POSITION]->(n);

MATCH (h:Hometown)
SET h.location = point({latitude: toFloat(h.hometown_lat), longitude: toFloat(h.hometown_long)})

MATCH (c:College)-[:COMMITTED_TO]->(n:Player)
WITH n, c 
MATCH (c:College)-[:COMMITTED_TO]->(n)<-[:IS_FROM]-(h:Hometown)
WITH count(n) as e, c, h
CREATE (c)-[r:HAS_PIPELINE]->(h)
SET r.frequency = e
RETURN type(r), r.frequency

MATCH (c1:College)-[r:HAS_PIPELINE]->(h:Hometown)<-[:HAS_PIPELINE]-(c2:College)
WITH c1, c2, count(r) as Cnt
WHERE c1.name <> 'Uncommitted' AND c2.name <> 'Uncommitted'
CREATE (c1)-[r:SHARES_HOMETOWN]->(c2)
SET r.city_count = Cnt
RETURN type(r), r.city_count

MATCH (h1:Hometown {state: 'NC'}),(h2:Hometown {state: 'NC'})  
WHERE id(h1) < id(h2) 
WITH h1,h2, distance(h1.location,h2.location) as distance 
WHERE distance < 35000  
MERGE (h1)-[l:COST]->(h2) ON CREATE SET l.distance = distance

SyntaxError: invalid syntax (<ipython-input-37-6646e03cc113>, line 1)